# Build a Qusetion Answering Engine in Minutes

This notebook illustrates how to build a question answering engine from scratch using [Milvus](https://milvus.io/) and [Towhee](https://towhee.io/). Milvus is the most advanced open-source vector database built for AI applications and supports nearest neighbor embedding search across tens of millions of entries, and Towhee is a framework that provides ETL for unstructured data using SoTA machine learning models.

We will go through question answering procedures and evaluate performance. Moreover, we managed to make the core functionality as simple as almost 10 lines of code with Towhee, so that you can start hacking your own question answering engine.

## Preparations

### Install Dependencies

First we need to install dependencies such as pymilvus, towhee and gradio.

In [ ]:
! python -m pip install -q pymilvus towhee gradio

### Prepare the Data

There is a subset of the  [InsuranceQA Corpus](https://github.com/shuzi/insuranceQA)  (1000 pairs of questions and answers) used in this demo, everyone can download on [Github](https://github.com/towhee-io/examples/releases/download/data/question_answer.csv).

In [2]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/question_answer.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  595k  100  595k    0     0   433k      0  0:00:01  0:00:01 --:--:--  433k


**question_answer.csv**: a file containing question and the answer.

Let's take a quick look:

In [1]:
import pandas as pd

df = pd.read_csv('question_answer.csv')
df.head()



,id,question,answer
0,0,Is Disability Insurance Required By Law?,Not generally. There are five states that requ...
1,1,Can Creditors Take Life Insurance After ...,If the person who passed away was the one with...
2,2,Does Travelers Insurance Have Renters Ins...,One of the insurance carriers I represent is T...
3,3,Can I Drive A New Car Home Without Ins...,Most auto dealers will not let you drive the c...
4,4,Is The Cash Surrender Value Of Life Ins...,Cash surrender value comes only with Whole Lif...


In [9]:
df["id"] = df.index

In [10]:
df = df[["id", "question", "answer"]]

In [17]:
df.to_csv('question_answer3.csv', index=False)

To use the dataset to get answers, let's first define the dictionary:

- `id_answer`: a dictionary of id and corresponding answer

In [2]:
id_answer = df.set_index('id')['answer'].to_dict()

### Create Milvus Collection

Before getting started, please make sure you have [installed milvus](https://milvus.io/docs/v2.0.x/install_standalone-docker.md). Next to define the function `create_milvus_collection` to create collection in Milvus that uses the [L2 distance metric](https://milvus.io/docs/v2.0.x/metric.md#Euclidean-distance-L2) and an [IVF_FLAT index](https://milvus.io/docs/v2.0.x/index.md#IVF_FLAT).

In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection



## Question Answering Engine

In this section, we will show how to build our question answering engine using Milvus and Towhee. The basic idea behind question answering is to use Towhee to generate embedding from the question dataset and compare the input question with the embedding stored in Milvus.

[Towhee](https://towhee.io/) is a machine learning framework that allows the creation of data processing pipelines, and it also provides predefined operators for implementing insert and query operations in Milvus.

<img src="./workflow.png" width = "60%" height = "60%" align=center />

### Load question embedding into Milvus

We first generate embedding from question text with [dpr](https://towhee.io/text-embedding/dpr) operator and insert the embedding into Milvus. Towhee provides a [method-chaining style API](https://towhee.readthedocs.io/en/main/index.html) so that users can assemble a data processing pipeline with operators.

In [4]:
import towhee

/Users/Michael/rikai/chatbot/milvus/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
%pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00m eta 0:00:01:01
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
%%time

collection = create_milvus_collection('question_answer', 768)

dc = (
    towhee.read_csv('question_answer.csv')
      .runas_op['id', 'id'](func=lambda x: int(x))
      .text_embedding.dpr['question', 'vec'](model_name="facebook/dpr-ctx_encoder-single-nq-base")
      .runas_op['vec', 'vec'](func=lambda x: x.squeeze(0))
      .tensor_normalize['vec', 'vec']()
      .to_milvus['id', 'vec'](collection=collection, batch=100)
)

2022-12-07 10:11:24,460 - 140704720311488 - decorators.py-decorators:71 - WARNING: [has_collection] retry:4, cost: 0.27s, reason: <_MultiThreadedRendezvous: StatusCode.UNAVAILABLE, internal: Milvus Proxy is not ready yet. please wait>
2022-12-07 10:11:24,737 - 140704720311488 - decorators.py-decorators:71 - WARNING: [has_collection] retry:5, cost: 0.81s, reason: <_MultiThreadedRendezvous: StatusCode.UNAVAILABLE, internal: Milvus Proxy is not ready yet. please wait>
2022-12-07 10:11:25,555 - 140704720311488 - decorators.py-decorators:71 - WARNING: [has_collection] retry:6, cost: 2.43s, reason: <_MultiThreadedRendezvous: StatusCode.UNAVAILABLE, internal: Milvus Proxy is not ready yet. please wait>
2022-12-07 10:11:27,992 - 140704720311488 - decorators.py-decorators:71 - WARNING: [has_collection] retry:7, cost: 7.29s, reason: <_MultiThreadedRendezvous: StatusCode.UNAVAILABLE, internal: Milvus Proxy is not ready yet. please wait>
2022-12-07 10:11:35,286 - 140704720311488 - decorators.py-de

CPU times: user 2min 57s, sys: 8.71 s, total: 3min 5s
Wall time: 1min 19s


In [6]:
print('Total number of inserted data is {}.'.format(collection.num_entities))

Total number of inserted data is 100.


#### Explanation of Data Processing Pipeline

Here is detailed explanation for each line of the code:

`towhee.read_csv('question_answer.csv')`: read tabular data from the file (id, question and answer columns);

`.runas_op['id', 'id'](func=lambda x: int(x))`: for each row from the data, convert the data type of the column id from `str` to `int`;

`.text_embedding.dpr['question', 'vec'](model_name="facebook/dpr-ctx_encoder-single-nq-base")`: use the `acebook/dpr-ctx_encoder-single-nq-base` model to generate the question embedding vector with the [dpr operator](https://towhee.io/text-embedding/dpr) in towhee hub;

`.runas_op['vec', 'vec'](func=lambda x: x.squeeze(0))`: the vec shape after dpr operator is (1, 768), so we need to squeeze it;

`.tensor_normalize['vec', 'vec']()`: normalize the embedding vector;

`.to_milvus['id', 'vec'](collection=collection, batch=100)`: insert question embedding vector into Milvus;

Then we decompose the pipeline into stages to get a more intuitive understanding of what's going on

- read data from csv, get a tabular data with two columns

In [7]:
towhee.read_csv('question_answer.csv').show()

id,question,answer
0,Is Disability Insurance Required By Law?,Not generally. There are five states that require most all employers carry short term disability insurance on their employees. T...
1,Can Creditors Take Life Insurance After Death?,"If the person who passed away was the one with the debt, creditors generally cannot take the life insurance proceeds left as lon..."
2,Does Travelers Insurance Have Renters Insurance?,"One of the insurance carriers I represent is Travelers and yes, you can purchase Renters insurance through Travelers. I would lo..."
3,Can I Drive A New Car Home Without Insurance?,Most auto dealers will not let you drive the car off of the lot unless you have proof of insurance if there is a loan on the car...
4,Is The Cash Surrender Value Of Life Insurance Taxable?,"Cash surrender value comes only with Whole Life Insurance, never with Term. It is the amount you can terminate your policy and r..."


- add `runas_op` to convert the data type

In [18]:
dc1 = towhee.read_csv('question_answer.csv').head(2).to_list()
dc2 = towhee.read_csv('question_answer.csv').runas_op['id', 'id'](func=lambda x: int(x)).head(2).to_list()
print(dc1[0].id, dc2[0].id, type(dc1[0].id), type(dc2[0].id))

0 0 <class 'str'> <class 'int'>


- generate embedding with `vec` column

In [9]:
%%time
towhee.read_csv('question_answer.csv') \
      .head(2) \
      .text_embedding.dpr['question', 'vec'](model_name="facebook/dpr-ctx_encoder-single-nq-base") \
      .show()

id,question,answer,vec
0,Is Disability Insurance Required By Law?,Not generally. There are five states that require most all employers carry short term disability insurance on their employees. T...,"[0.7798514, 0.139864, 0.26759088, ...] shape=(1, 768)"
1,Can Creditors Take Life Insurance After Death?,"If the person who passed away was the one with the debt, creditors generally cannot take the life insurance proceeds left as lon...","[0.093207195, -0.14792548, 0.24360597, ...] shape=(1, 768)"


CPU times: user 1.89 s, sys: 1.46 s, total: 3.35 s
Wall time: 5.21 s


### Ask Question with Milvus and Towhee

Now that embedding for question dataset have been inserted into Milvus, we can ask question with Milvus and Towhee. Again, we use Towhee to load the input question, compute a embedding, and use it as a query in Milvus. Because Milvus only outputs IDs and distance values, we provide the `id_answers` dictionary to get the answers based on IDs and display.

In [37]:
%%time
dc = ( towhee.dc(['Is  Disability  Insurance  Required  By  Law?'])
      .text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base")
      .runas_op(func=lambda x: x.squeeze(0))
      .tensor_normalize()
      .milvus_search(collection='question_answer', limit=1)
      .runas_op(func=lambda res: [x.score for x in res])
      .to_list()
)

CPU times: user 1.76 s, sys: 439 ms, total: 2.2 s
Wall time: 3.3 s


Then we can get the answer about 'Is  Disability  Insurance  Required  By  Law?'.

In [38]:
dc[0][0]

0.0

## Release a Showcase

We've done an excellent job on the core functionality of our question answering engine. Now it's time to build a showcase with interface. [Gradio](https://gradio.app/) is a great tool for building demos. With Gradio, we simply need to wrap the data processing pipeline via a `chat` function:

In [45]:
def chat(message, history):
    history = history or []
    with towhee.api() as api:
           qa_function = (
                api.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base")
                    .runas_op(func=lambda x: x.squeeze(0))
                    .tensor_normalize()
                    .milvus_search(collection='question_answer', limit=3)
                    .runas_op(func=lambda res: [id_answer[x.id]+'\n' for x in res])
                    .as_function()
            )
    response = qa_function(message)[0]
    history.append((message, response))
    return history, history

In [ ]:
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    price: float
    is_offer: Union[bool, None] = None


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/items/{item_id}")
def read_item(item_id: int, message: Union[str, None] = None):
    
    return {"item_id": item_id, "q": }


@app.put("/items/{item_id}")
def update_item(item_id: int, item: Item):
    return {"item_name": item.name, 'item_id': item_id}


In [51]:
def chat(message):
   
    with towhee.api() as api:
           qa_function = (
                api.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base")
                    .runas_op(func=lambda x: x.squeeze(0))
                    .tensor_normalize()
                    .milvus_search(collection='question_answer', limit=3)
                    .runas_op(func=lambda res: [id_answer[x.id]+'\n' for x in res])
                    .as_function()
            )
    response = qa_function(message)[0]
  
    return response

In [52]:
chat('Is  Disability  Insurance  Required  By  Law?')

'Not generally. There are five states that require most all employers carry short term disability insurance on their employees. These states are: California, Hawaii, New Jersey, New York, and Rhode Island. Besides this mandatory short term disability law, there is no other legislative imperative for someone to purchase or be covered by disability insurance.\n'

In [46]:
import gradio

chatbot = gradio.Chatbot(color_map=("green", "gray"))
interface = gradio.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
interface.launch(inline=True, share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://b0aa6f7f439f76a8.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
